# Decision Trees

In previous chapters, we explored classifiers that rely on mathematical optimization. In this notebook, we'll take a different route and introduce a model that mimics human reasoning: **decision trees**.

Rather than relying on numerical optimization, decision trees build a sequence of rules—such as *"If an apple is red and large, then it is good"*—to classify data points. This logic-based approach is intuitive and can be visualized as a flow of decisions, which makes decision trees particularly useful for both learning and interpreting classification tasks.

## 1.1 Toy Example

To understand decision trees intuitively, let's begin with a toy dataset. Imagine we want to classify apples as **good** or **bad** based on three binary features:

- **Size**: large or small  
- **Color**: red or green  
- **Spots**: present or absent

Each apple also comes with a label: whether it is good or bad. Our goal is to discover decision rules such as:

> “If an apple is large and green, then it is good.”

This is precisely the kind of logic that decision trees aim to formalize: a sequence of feature-based rules that result in a classification.

In [ ]:
from IPython.display import Image
Image('../illustrations/apple_tree.png', width=900)

In this image, each apple is described by its features and labeled as either *Good* or *Bad*. Given this, we can try to identify patterns:

- Are large apples more likely to be good?
- Does color play a role?
- Do spots indicate bad quality?

While this is manageable by eye for a small dataset, things get complex when we have many features or thousands of samples. This is where decision trees shine: they help us **automate the discovery of such rules**.

## 1.2 How to Formalize Rules

As humans, we often make hypotheses based on a few examples and test whether these rules generalize. However, when dealing with large datasets and many features, we need a **systematic** way to generate and evaluate these rules.

This is exactly what decision trees do. Their core logic follows a simple procedure:

1. **Test multiple feature-based questions** to split the data.
2. **Select the most informative split**.
3. **Repeat the process** recursively on each subset until all items are classified or a stopping criterion is met.

Let’s explore how this works in practice.

### 1.2.1 Creating Splits

For any feature, we can ask a **yes/no question** to split our dataset into two groups. For example:

- “Is the apple red?”
- “Is the apple large?”

Each question creates a **binary split**, helping us organize the data into smaller, hopefully more homogeneous groups. 

If we ask *"Is the apple red?"*, we get the following two groups for the answers no and yes:

In [ ]:
Image('../illustrations/apple_tree_split.png', width=400)

Alternatively, if we ask *"is the apple large?"*, we obtain the following two groups:

In [ ]:
Image('../illustrations/apple_tree_split2.png', width=500)

We can see that neither of these simple questions perfectly separates the apples into good and bad categories—both splits still contain a mix of labels.

To build an effective decision tree, we need to **evaluate all possible splits** and choose the one that results in the **most distinct separation** between labels.

So, how do we quantify the "best" split? One common approach is to measure how *pure* each resulting group is.

### 1.2.2 Best Splits and Entropy

A good split separates the data into groups that are as **pure** as possible—that is, each group contains items of mostly one label. One way to measure this purity is using a concept from information theory called **entropy**.

Entropy quantifies how mixed a subset is. In physics, entropy measures disorder; in classification, it tells us how uncertain or impure a group is.

For a binary classification task, entropy is defined as:

$$
E = -p_0 \log(p_0) - p_1 \log(p_1)
$$

where:
- $p_0$ is the fraction of one class (e.g., "bad" apples),
- $p_1 = 1 - p_0$ is the fraction of the other class (e.g., "good" apples).

Entropy reaches its maximum when both classes are equally mixed ($p_0 = 0.5$), and it is zero when all elements belong to the same class.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from PIL import Image as PILImage

def entropy(x):
    return -x * np.log(x) - (1 - x) * np.log(1 - x)

fig, ax = plt.subplots(figsize=(10, 5))
x_vals = np.arange(0.01, 1, 0.01)
ax.plot(x_vals, entropy(x_vals), 'k')
ax.set_xlabel(r"$p_{\text{red}}$", fontsize=16)
ax.set_ylabel("Entropy", fontsize=16)
ax.set_ylim(0.1, 1.1)
ax.set_xlim(-0.8, 1.8)
ax.set_xticks([0, 0.5, 1])

positions = ["lower left", "upper center", "lower right"]
for i, pos in enumerate(positions):
    im = PILImage.open(f"../illustrations/entropy{i+1}.png")
    axins = inset_axes(ax, width="30%", height="40%", loc=pos)
    axins.imshow(im)
    axins.set_axis_off()

plt.show();

As shown in the plot above, entropy is highest when the dataset is perfectly mixed (e.g., 50% red, 50% blue), and lowest when the dataset is pure (e.g., all red or all blue).

This gives us a simple rule:  
> **The lower the entropy after a split, the better the split.**

In practice, we compute the entropy for every possible feature-based split and choose the one that produces the **purest** subsets — i.e., the lowest combined entropy.

### 1.2.3 Information Gain

While entropy measures how mixed a group is, what we actually care about is how much **better** a split makes our classification.

This is captured by **information gain (IG)**, which tells us how much entropy is **reduced** by splitting the data using a specific feature.

The formula for information gain is:

$$
IG = E_{\text{parent}} - \left( \frac{n_1}{N} E_1 + \frac{n_2}{N} E_2 \right)
$$

Where:
- $E_{\text{parent}}$ is the entropy before the split,
- $E_1$, $E_2$ are the entropies of the two resulting subsets,
- $n_1$, $n_2$ are their respective sizes, and
- $N = n_1 + n_2$ is the size of the parent set.

This formula gives higher scores to splits that:
- Greatly reduce entropy, and
- Affect a large portion of the data.

Splits that isolate a single point (very small $n_1$ or $n_2$) typically have **low** information gain and are therefore avoided.

## 1.3 Splitting Apples

Let’s now apply the above ideas to our apple dataset.

Suppose we compute the information gain for all features on the initial dataset. We find that the most informative one is **color**. This leads to the first split:

> "Is the apple green?"

This split separates the apples into two groups. From there, we repeat the process recursively on each group, always choosing the next most informative feature.

In [ ]:
Image('../illustrations/apple_first_split.png', width=600)

## 1.4 Recursive Splitting

After the first split, we now treat each resulting subset as a smaller classification problem.

For the **left subset** (e.g., green apples), the most informative feature might be **size**:
> "Is the apple large?"

This results in perfectly pure groups: one contains only good apples, the other only bad ones.

In [ ]:
Image('../illustrations/apple_second_split_1.png', width=500)

For the **right subset** (e.g., red apples), the best feature might be **spots**:
> "Does the apple have spots?"

Again, this creates completely pure groups.

At this point, all examples are perfectly classified — and the tree is complete!

In [ ]:
Image('../illustrations/apple_second_split_2.png', width=500)

## 1.5 The Final Tree

After applying all the splits, we can represent the entire classification process as a **decision tree**.

Each internal node corresponds to a question about a feature.  
Each leaf node gives us a classification result (e.g., good or bad apple).

This tree can now be used to classify **new** apples: we simply follow the branches based on their features until we reach a decision.

In [ ]:
Image('../illustrations/apple_decision_tree.png', width=500)

### 1.5.1 Non-Categorical Variables

In our example, we only used **categorical** features (e.g., size = large/small, color = red/green).  
However, decision trees can easily handle **continuous** variables as well.

Instead of splitting based on discrete categories, the tree chooses a **threshold**:

> “Is the apple’s size greater than 8 cm?”

This kind of question also creates two groups, and the tree selects the **best threshold** by maximizing the information gain — just as before.

This flexibility makes decision trees suitable for both numerical and categorical data.

## 1.6 Decision Trees in scikit-learn

Let’s now build a real decision tree using `scikit-learn`.

We’ll work with the **seeds dataset** from the UCI repository, which contains several measurements of wheat kernels. Each row describes a seed using features like area, perimeter, length, and groove length, and includes a label indicating the seed type.

We’ll begin by importing the relevant class:

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

# Load dataset from UCI repository
seeds = pd.read_csv(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/00236/seeds_dataset.txt',
    sep='\t',
    on_bad_lines='skip',
    names=['area', 'perimeter', 'compactness', 'length', 'width',
           'symmetry_coef', 'length_groove', 'seed_type']
)

seeds.head()

In [ ]:
# Define feature matrix X and target vector y
X = seeds[['area', 'perimeter', 'compactness', 'length', 'width',
           'symmetry_coef', 'length_groove']]
y = seeds['seed_type']

In [ ]:
# Create a decision tree classifier using entropy as the splitting criterion
tree_clf = DecisionTreeClassifier(criterion='entropy')

# Train the model on the data
tree_clf.fit(X, y)

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

# Plot the trained decision tree
fig, ax = plt.subplots(figsize=(20, 10))
plot_tree(tree_clf, filled=True, ax=ax, feature_names=X.columns)
plt.title("Decision Tree for Seeds Dataset")
plt.show()

Each node in the tree corresponds to a question like:

> “Is groove length less than 5.5?”

The dataset is recursively split based on these questions. Each node displays:
- The condition used to split
- The entropy value
- The number of samples
- The sample distribution across classes/categories

The **leaf nodes** (bottom boxes) represent final predictions. Their entropy is often 0, indicating pure subsets.

The **colors** reflect the predicted class, and the color intensity shows how pure the node is.  
We can also observe that some features, like `length_groove`, are used multiple times—showing their importance in classification.

### 1.6.1 Adjusting Tree Parameters

In our earlier tree, some leaves contain only a single sample. This indicates **overfitting**: the model tries too hard to perfectly fit the training data.

To avoid overfitting, we can control the tree’s complexity by setting parameters such as:

- `max_depth`: limits the number of decision levels in the tree
- `min_samples_leaf`: sets the minimum number of samples per leaf node

Let’s see what happens when we limit the tree depth.

In [ ]:
# Limit the depth of the tree to prevent overfitting
tree_clf = DecisionTreeClassifier(max_depth=2, criterion='entropy')
tree_clf.fit(X, y)

# Plot the simplified tree
fig, ax = plt.subplots(figsize=(20, 10))
plot_tree(tree_clf, filled=True, ax=ax, feature_names=X.columns)
plt.title("Decision Tree with max_depth=2")
plt.show()

By limiting the tree depth to 2, we now have a much **simpler model**.

- The tree is easier to interpret.
- It generalizes better to unseen data.
- However, it may not classify the training data as accurately as a deeper tree.

We can also see that the **leaf nodes now have non-zero entropy**, meaning the splits are no longer perfectly pure — and that's okay! Simpler trees often perform better on new data.

Alternatively we can also specify a minimum number of elements per split:

In [ ]:
# Require at least 10 samples per leaf to reduce overfitting
tree_clf = DecisionTreeClassifier(min_samples_leaf=10, criterion='entropy')
tree_clf.fit(X, y)

# Plot the resulting tree
fig, ax = plt.subplots(figsize=(20, 10))
plot_tree(tree_clf, filled=True, ax=ax, feature_names=X.columns)
plt.title("Decision Tree with min_samples_leaf=10")
plt.show()

Setting `min_samples_leaf=10` means that each final group must contain **at least 10 samples**.

This prevents the tree from splitting on **tiny patterns** that may not generalize well, and helps reduce overfitting. As a result:

- Some potentially pure splits are avoided.
- The model trades off a bit of accuracy for better generalization.

Just like with `max_depth`, this is part of **regularization**—controlling model complexity to balance training accuracy and test performance.

## 1.7 Random Forests

A **random forest** is an ensemble method that builds not one, but many decision trees, and then combines their predictions.

Instead of relying on a single tree (which might overfit), random forests:
- Train multiple trees on **random subsets** of the data (with replacement),
- Use **random subsets of features** for each split,
- Aggregate the predictions (e.g., by majority vote).

This makes them more robust, often achieving better accuracy and generalization. One advantage of this method is that it can find interesting patterns existing in subsets of the data, as the model doesn't have to satisfy all constraints at the same time.

Let’s try it out in `scikit-learn`.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create a random forest classifier with 50 trees
rf_model = RandomForestClassifier(n_estimators=50)
rf_model.fit(X, y)

Since we now learned 50 different trees, we cannot visualize them anymore. However, Random Forests can give us insights into the importance of each feature in our dataset:

In [ ]:
# Show feature importances
import pandas as pd

pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_  # Stores information about feature importance
}).sort_values(by='importance', ascending=False)

The table above shows how much each feature contributed to the forest's decision-making process.

- Higher importance values mean that the feature was used more often, and more effectively, to split the data.
- In this example, we can see that `length_groove` is the most informative feature—just like in the single decision tree!

Unlike a single tree, a random forest doesn’t produce an interpretable decision path, but it often results in **higher accuracy** and **better generalization**.


## Exercise: Predict House Quality with a Decision Tree

Let’s now apply what we’ve learned on a real-world dataset!

Use the **King County housing dataset** `kc_house_data.csv`, which includes house features and sale prices.

### Tasks:

1. Import the dataset

2. Create a new binary column `good_bad`:  Set it to 1 if `grade > 7`, else 0.

3. Remove outliers:  Keep only rows where `sqft_living < 8000` and `bedrooms < 10`.

4. Split the dataset into training and test sets  (e.g., 80% training, 20% testing)

5. Train a `DecisionTreeClassifier` using these features: `price`, `bedrooms`, `bathrooms`, `sqft_living`, `sqft_lot`, `floors`. Use entropy as the criterion and a max depth of three.

6. What is the most informative feature (top of the tree)?

7. Evaluate model accuracy.

8. Increase `max_depth` to 25, retrain, and re-evaluate. Does accuracy improve?